In [ ]:
# ====================================================================
#        GRÁFICO DE INFLUENCIADORES - DBO (5, 20) | 2012 a 2020
# ====================================================================

import pandas as pd
import plotly.express as px

# Caminho relativo do arquivo Excel (coloque o arquivo na pasta 'dados' do repositório)
arquivo = 'dados/TIET04200.xlsx'

# Leitura da aba "Analise"
df = pd.read_excel(arquivo, sheet_name="Analise")

# Converter coluna Data para datetime e filtrar entre 2012 e 2020
df['Data'] = pd.to_datetime(df['Data'], dayfirst=True)
df = df[(df['Data'] >= '2012-01-01') & (df['Data'] <= '2019-12-31')]

# Pivotar para ter Data no índice e parâmetros nas colunas
df_pivot = df.pivot(index='Data', columns='Parametro', values='Valor')

# Calcular correlação com o parâmetro alvo "DBO (5, 20)"
corr = df_pivot.corr()['DBO (5, 20)'].drop('DBO (5, 20)').dropna()

# Preparar DataFrame para o gráfico
df_corr = corr.reset_index()
df_corr.columns = ['Parametro', 'Correlacao']

# Plot interativo com Plotly Express
fig = px.bar(
    df_corr,
    x='Parametro',
    y='Correlacao',
    title='Correlação dos Parâmetros com DBO (5, 20) — Período: 2012 a 2019',
    labels={'Correlacao': 'Coeficiente de Correlação', 'Parametro': 'Parâmetro'},
    color='Correlacao',
    color_continuous_scale='Viridis',
    text=df_corr['Correlacao'].apply(lambda x: f"{x:.2f}")
)

fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_tickangle=-45,
    uniformtext_minsize=10,
    uniformtext_mode='hide',
    yaxis=dict(range=[min(df_corr['Correlacao']) - 0.1, 1])
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Caminho relativo do arquivo Excel (coloque o arquivo na pasta 'dados' do repositório)
arquivo = 'dados/TIET04200.xlsx'
df = pd.read_excel(arquivo, sheet_name="Analise")

# Converter coluna Data para datetime e filtrar entre 2012 e 2019
df['Data'] = pd.to_datetime(df['Data'], dayfirst=True)
df = df[(df['Data'] >= '2012-01-01') & (df['Data'] <= '2019-12-31')]

# Pivotar para ter Data no índice e parâmetros nas colunas
df_pivot = df.pivot(index='Data', columns='Parametro', values='Valor')

# Calcular correlação com o parâmetro alvo "DBO (5, 20)"
corr = df_pivot.corr()['DBO (5, 20)'].drop('DBO (5, 20)').dropna()

# Pegar só os 6 maiores em valor absoluto
corr_top6 = corr.reindex(corr.abs().sort_values(ascending=False).index)[:6]

# Preparar DataFrame para o gráfico
df_corr = corr_top6.reset_index()
df_corr.columns = ['Parametro', 'Correlacao']

# Ajustar range_color para seu intervalo real
min_corr = df_corr['Correlacao'].min()
max_corr = df_corr['Correlacao'].max()

# Definir escala só com azul, do clarinho (#cce5ff) até azul escuro (#003366)
blue_scale = ['#cce5ff', '#003366']

fig = px.bar(
    df_corr,
    x='Parametro',
    y='Correlacao',
    title='Top 6 Parâmetros com Maior Correlação com DBO (5, 20) — 2012 a 2019',
    labels={'Correlacao': 'Coeficiente de Correlação', 'Parametro': 'Parâmetro'},
    color='Correlacao',
    color_continuous_scale=blue_scale,
    range_color=[min_corr, max_corr],
    text=df_corr['Correlacao'].apply(lambda x: f"{x:.2f}")
)

fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_tickangle=-45,
    uniformtext_minsize=10,
    uniformtext_mode='hide',
    yaxis=dict(range=[min_corr - 0.1, 1])
)

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Caminho relativo do arquivo Excel (coloque o arquivo na pasta 'dados' do repositório)
arquivo = 'dados/TIET04200.xlsx'
df = pd.read_excel(arquivo, sheet_name="Analise")

# Converter coluna Data para datetime e filtrar entre 2012 e 2020
df['Data'] = pd.to_datetime(df['Data'], dayfirst=True)
df_periodo = df[(df['Data'] >= '2012-01-01') & (df['Data'] <= '2019-12-31')]

# Lista dos parâmetros que quer analisar
parametros = ['DBO (5, 20)', 'Fósforo Total', 'Condutividade', 'Sólido Dissolvido Total', 'Nitrogênio Amoniacal', 'Sólido Total']

# Dicionário para armazenar as estatísticas (sem Contagem, Assimetria, Curtose e Valores Nulos)
estatisticas_geral = {
    'Parâmetro': [],
    'Média': [],
    'Mediana': [],
    'Desvio Padrão': [],
    'Mínimo': [],
    'Máximo': [],
    '1º Quartil (25%)': [],
    '3º Quartil (75%)': []
}

for p in parametros:
    dados = df_periodo[df_periodo['Parametro'] == p]['Valor']
    estatisticas_geral['Parâmetro'].append(p)
    estatisticas_geral['Média'].append(dados.mean())
    estatisticas_geral['Mediana'].append(dados.median())
    estatisticas_geral['Desvio Padrão'].append(dados.std())
    estatisticas_geral['Mínimo'].append(dados.min())
    estatisticas_geral['Máximo'].append(dados.max())
    estatisticas_geral['1º Quartil (25%)'].append(dados.quantile(0.25))
    estatisticas_geral['3º Quartil (75%)'].append(dados.quantile(0.75))

# Transformar em DataFrame
df_stats = pd.DataFrame(estatisticas_geral)

# Formatar números decimais para duas casas, exceto parâmetro
for col in df_stats.columns:
    if col != 'Parâmetro':
        df_stats[col] = df_stats[col].apply(lambda x: f"{x:.2f}" if pd.notnull(x) else "NaN")

# Criar tabela Plotly interativa
fig = go.Figure(data=[go.Table(
    columnwidth=[150] + [110]*7,
    header=dict(
        values=[f"<b>{c}</b>" for c in df_stats.columns],
        fill_color='#4B8BBE',
        font=dict(color='white', size=14),
        align='center',
        height=40
    ),
    cells=dict(
        values=[df_stats[col] for col in df_stats.columns],
        fill_color=[['#D6EAF8', 'white'] * (len(df_stats) // 2 + 1)],
        align='center',
        font=dict(color='#222222', size=12),
        height=30
    )
)])

fig.update_layout(
    title_text='Estatísticas dos Parâmetros (2012 - 2020)',
    title_font_size=18,
    title_x=0.5,
    width=1200,
    height=500,
    margin=dict(l=20, r=20, t=60, b=20)
)

fig.show()


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates  # para formatar datas

from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ==================== config ====================
ARQUIVO = "dados/TIET04200.xlsx"
SHEET   = "Analise"
DATECOL = "Data"
TARGET  = "DBO (5, 20)"
BASE_FEATS = ['Fósforo Total', 'Condutividade', 'Sólido Dissolvido Total', 'Nitrogênio Amoniacal', 'Sólido Total']

N_RECENT_FOLDS = 3      # quantos folds recentes considerar
ROLL_WINDOW    = 6
OUT_DIR        = "resultados_artigo"
os.makedirs(OUT_DIR, exist_ok=True)

# ==================== utils ====================
def mape(y_true, y_pred):
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)
    mask = y_true != 0
    if not np.any(mask):
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def get_estacao(ts: pd.Timestamp):
    ano = ts.year
    verao_i = pd.Timestamp(ano, 12, 21)
    outono_i = pd.Timestamp(ano, 3, 21)
    inverno_i = pd.Timestamp(ano, 6, 21)
    primav_i = pd.Timestamp(ano, 9, 23)
    verao_f = pd.Timestamp(ano, 3, 20)
    if (ts >= verao_i) or (ts <= verao_f): return "Verao"
    elif (ts > outono_i) and (ts <= inverno_i): return "Outono"
    elif (ts > inverno_i) and (ts <= primav_i): return "Inverno"
    else: return "Primavera"

# ==================== dados ====================
if not os.path.exists(ARQUIVO):
    raise FileNotFoundError(ARQUIVO)

df = pd.read_excel(ARQUIVO, sheet_name=SHEET)
if DATECOL not in df.columns:
    raise ValueError(f"Coluna '{DATECOL}' não encontrada.")
df[DATECOL] = pd.to_datetime(df[DATECOL], dayfirst=True)
df = df[(df[DATECOL] >= "2012-01-01") & (df[DATECOL] <= "2019-12-31")]

if not {"Parametro","Valor"}.issubset(df.columns):
    raise ValueError("Faltam colunas Parametro/Valor.")

pivot = df.pivot(index=DATECOL, columns="Parametro", values="Valor").sort_index()
pivot = pivot.drop(columns=[c for c in ['Nitrogênio Total','Dureza','Sólido Suspenso Total'] if c in pivot.columns],
                   errors='ignore')
if TARGET not in pivot.columns:
    raise ValueError(f"Alvo '{TARGET}' não está na planilha.")

features = [c for c in BASE_FEATS if c in pivot.columns]

dm = pd.DataFrame(index=pivot.index)
dm[TARGET] = pivot[TARGET]
for c in features:
    dm[c] = pivot[c]

dm['estacao'] = dm.index.to_series().apply(get_estacao)
dm = pd.get_dummies(dm, columns=['estacao'])
dm['mes'] = dm.index.month
dm['dia_do_ano'] = dm.index.dayofyear

# lags/rollings (nowcasting: só passado)
dm['DBO_lag1'] = dm[TARGET].shift(1)
dm['DBO_lag2'] = dm[TARGET].shift(2)
dm['DBO_lag3'] = dm[TARGET].shift(3)
dm['DBO_roll_mean3'] = dm[TARGET].rolling(3).mean().shift(1)
dm['DBO_roll_std3']  = dm[TARGET].rolling(3).std().shift(1)

dm = dm.dropna(subset=[TARGET,'DBO_lag1','DBO_lag2','DBO_lag3','DBO_roll_mean3','DBO_roll_std3'])

X = dm.drop(columns=[TARGET])
y = dm[TARGET]

# ==================== modelos base ====================
rf_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("rf", RandomForestRegressor(
        n_estimators=100, max_depth=24, max_features='sqrt',
        min_samples_leaf=3, min_samples_split=6, random_state=42
    ))
])
mlp_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc",  StandardScaler()),
    ("mlp", MLPRegressor(hidden_layer_sizes=(100,50), activation='tanh',
                         solver='lbfgs', alpha=0.00395, max_iter=5000, random_state=42))
])
BASES = [rf_pipe, mlp_pipe]

# ==================== stacking OOF (sem vazamento) ====================
def build_oof_blocks(n, min_train=12, n_blocks=5, min_block_size=2):
    if n <= min_train + 1:
        return [(np.arange(0, max(1,n-1)), np.arange(max(1,n-1), n))]
    remain = n - min_train
    nb = max(1, min(n_blocks, remain // max(1,min_block_size)))
    base = remain // nb; extra = remain % nb
    sizes = [base + (1 if i < extra else 0) for i in range(nb)]
    splits=[]; cur=min_train
    for s in sizes:
        splits.append((np.arange(0,cur), np.arange(cur, cur+s)))
        cur += s
    return splits

def train_meta_with_oof(Xtr, ytr, bases, meta_kind="ridge", ridge_alpha=1.0):
    n = len(Xtr)
    splits = build_oof_blocks(n, min_train=min(12, max(3, n//3)))
    oof = np.full((n, len(bases)), np.nan)
    for tr, te in splits:
        if len(tr)<1 or len(te)<1:
            continue
        for j, b in enumerate(bases):
            m = clone(b)
            m.fit(Xtr.iloc[tr], ytr.iloc[tr])
            oof[te, j] = m.predict(Xtr.iloc[te])
    valid = np.all(np.isfinite(oof), axis=1)
    if valid.sum() < 2:
        meta = None
    else:
        Xmeta = oof[valid]; ymeta = ytr.iloc[valid]
        meta = Ridge(alpha=ridge_alpha) if meta_kind=="ridge" else LinearRegression()
        meta.fit(Xmeta, ymeta)
    fitted = [clone(b).fit(Xtr, ytr) for b in bases]
    return meta, fitted

def stacked_predict(meta, fitted, X_):
    preds = [m.predict(X_) for m in fitted]
    M = np.column_stack(preds)
    return M.mean(axis=1) if meta is None else meta.predict(M)

# ==================== CV externa (TimeSeriesSplit) ====================
tscv = TimeSeriesSplit(n_splits=5)
fold_rows = []
folds_info = []
y_pred_all = pd.Series(index=y.index, dtype=float)

for f, (tr_idx, te_idx) in enumerate(tscv.split(X), start=1):
    Xtr, Xte = X.iloc[tr_idx], X.iloc[te_idx]
    ytr, yte = y.iloc[tr_idx], y.iloc[te_idx]
    meta, fitted = train_meta_with_oof(Xtr, ytr, BASES, meta_kind="ridge", ridge_alpha=1.0)
    yhat = stacked_predict(meta, fitted, Xte)
    y_pred_all.iloc[te_idx] = yhat

    mse = mean_squared_error(yte, yhat)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(yte, yhat)
    r2  = r2_score(yte, yhat)
    mp  = mape(yte, yhat)
    fold_rows.append({"fold": f, "MSE": mse, "RMSE": rmse, "MAE": mae, "R2": r2, "MAPE": mp})

    folds_info.append({
        "fold": f,
        "test_idx": te_idx,
        "y_pred": pd.Series(yhat, index=y.index[te_idx]),
        "y_true": y.iloc[te_idx]
    })

df_metrics_all = pd.DataFrame(fold_rows).set_index("fold")
print("\n=== MÉTRICAS – TODOS OS FOLDS ===")
print(df_metrics_all.round(3))

# -------- últimos N folds --------
recent_folds = df_metrics_all.index[-N_RECENT_FOLDS:].tolist()
df_recent = df_metrics_all.loc[recent_folds]

# (A) média por fold (diagnóstico)
mean_by_fold = df_recent.mean().to_dict()

# (B) métricas pooled (concatenadas) — usar no gráfico e no artigo
y_pred_recent = pd.concat([f["y_pred"] for f in folds_info if f["fold"] in recent_folds]).sort_index()
y_true_recent = pd.concat([f["y_true"] for f in folds_info if f["fold"] in recent_folds]).sort_index()

pooled = {
    "MSE":  mean_squared_error(y_true_recent, y_pred_recent),
    "RMSE": mean_squared_error(y_true_recent, y_pred_recent) ** 0.5,
    "MAE":  mean_absolute_error(y_true_recent, y_pred_recent),
    "R2":   r2_score(y_true_recent, y_pred_recent),
    "MAPE": mape(y_true_recent, y_pred_recent),
}

print(f"\n=== ÚLTIMOS {N_RECENT_FOLDS} FOLDS — AGREGAÇÃO ===")
print("Média por fold:", {k: round(v, 3) for k, v in mean_by_fold.items()})
print("POOLED (concatenado):", {k: round(v, 3) for k, v in pooled.items()})
print("Folds usados:", recent_folds)

# ----- salvar tabela com folds + MEAN_FOLD + POOLED -----
recent_table = df_recent.copy()
recent_table.loc["MEAN_FOLD"] = recent_table.mean()
recent_table.loc["POOLED"] = pooled
recent_table.to_csv(os.path.join(OUT_DIR, f"metricas_ultimos_{N_RECENT_FOLDS}_folds.csv"), sep=";")
try:
    recent_table.to_excel(os.path.join(OUT_DIR, f"metricas_ultimos_{N_RECENT_FOLDS}_folds.xlsx"))
except Exception:
    pass

# ==================== rolling mean/std do alvo (contexto de regime) ====================
roll_mean = y.rolling(ROLL_WINDOW).mean()
roll_std  = y.rolling(ROLL_WINDOW).std()

plt.figure(figsize=(14,5))
plt.plot(y.index, y.values, label="DBO (valor observado)", linewidth=1.2)
plt.plot(roll_mean.index, roll_mean.values, label=f"Média móvel (janela={ROLL_WINDOW})", linewidth=2.0)
plt.plot(roll_std.index, roll_std.values, label=f"Desvio-padrão móvel (janela={ROLL_WINDOW})", linestyle="--")
plt.title("DBO — Média e Desvio-padrão móveis")
plt.xlabel("Data"); plt.ylabel("mg/L")
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))   # >>> rótulo a cada 3 meses
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))   # formato YYYY-MM
plt.xticks(rotation=45)
plt.legend(); plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "rolling_mean_std_pt.png"), bbox_inches="tight")
plt.show()

# ==================== Gráfico principal (PT) — métricas pooled ====================
start_recent = y_pred_recent.index.min()
end_recent   = y_pred_recent.index.max()

plt.figure(figsize=(16,6))
plt.plot(y_true_recent.index, y_true_recent.values, label='Real BDO (t)', marker='o', linestyle='-')
plt.plot(y_pred_recent.index, y_pred_recent.values, label='Prediction', marker='x', linestyle='--')

for fobj in folds_info:
    if fobj['fold'] in recent_folds:
        s = y.index[fobj['test_idx'][0]]
        e = y.index[fobj['test_idx'][-1]]
        plt.axvspan(s, e, color='gray', alpha=0.10)

titulo_pt = "DBO — Real vs. Previsto"
plt.title(titulo_pt)
plt.xlabel('Data'); plt.ylabel('DBO (mg/L)')
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))   # >>> trimestral
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=45)
plt.grid(True)

txt_pt = '\n'.join((
    f'Agregado (pooled) – últimos {N_RECENT_FOLDS} folds',
    f"MSE: {pooled['MSE']:.2f}",
    f"RMSE: {pooled['RMSE']:.2f}",
    f"MAE: {pooled['MAE']:.2f}",
    f"R2: {pooled['R2']:.3f}",
    f"MAPE: {pooled['MAPE']:.2f}%"
))
plt.gca().text(0.98, 0.98, txt_pt, transform=plt.gca().transAxes, fontsize=10,
               va='top', ha='right', bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.85))

plt.xlim([start_recent, end_recent])
plt.legend(); plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, f"nowcasting_ultimos_{N_RECENT_FOLDS}_folds_pt.png"), bbox_inches="tight")
plt.show()

# ==================== Gráfico principal (EN) — métricas pooled ====================
plt.figure(figsize=(16,6))
plt.plot(y_true_recent.index, y_true_recent.values, label='Observed', marker='o', linestyle='-')
plt.plot(y_pred_recent.index, y_pred_recent.values, label=f'Prediction (Last {N_RECENT_FOLDS} folds)', marker='x', linestyle='--')

for fobj in folds_info:
    if fobj['fold'] in recent_folds:
        s = y.index[fobj['test_idx'][0]]
        e = y.index[fobj['test_idx'][-1]]
        plt.axvspan(s, e, color='gray', alpha=0.10)

titulo_en = "BOD — Observed vs. Predicted"
plt.title(titulo_en)
plt.xlabel('Date'); plt.ylabel('BOD (mg/L)')
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))   # >>> trimestral
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=45)
plt.grid(True)

txt_en = '\n'.join((
    f'Pooled – last {N_RECENT_FOLDS} folds',
    f"MSE: {pooled['MSE']:.2f}",
    f"RMSE: {pooled['RMSE']:.2f}",
    f"MAE: {pooled['MAE']:.2f}",
    f"R2: {pooled['R2']:.3f}",
    f"MAPE: {pooled['MAPE']:.2f}%"
))
plt.gca().text(0.98, 0.98, txt_en, transform=plt.gca().transAxes, fontsize=10,
               va='top', ha='right', bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.85))

plt.xlim([start_recent, end_recent])
plt.legend(); plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, f"nowcasting_last_{N_RECENT_FOLDS}_folds_en.png"), bbox_inches="tight")
plt.show()
